In [11]:
# coding: utf-8

import numpy as np
np.random.seed(0)

class Data_Extractor:
    
    def __init__(self, raw_image, road_mask, img_size, pos_topleft_coord, neg_topleft_coord, normalization='mean', encoding='binary'):

        # original image info (shallow assignment to reduce mem)
        self.raw_image = raw_image
        self.road_mask = road_mask
        
        self.band = raw_image.shape[0]
    
        # patch info        
        self.img_size = img_size

        # batch info
        self.encoding = encoding # ground truth encoding
        self.normalization = normalization # if need norm

        # pos-neg coordinate info (array shallow assignment to reduce mem)
        self.pos_topleft_coord = pos_topleft_coord
        np.random.shuffle (self.pos_topleft_coord)
            
        self.neg_topleft_coord = neg_topleft_coord
        np.random.shuffle (self.neg_topleft_coord)
        
        self.pos_size = pos_topleft_coord.shape[0]
        self.neg_size = neg_topleft_coord.shape[0]

        self.pos_index = 0
        self.neg_index = 0
        self.index = 0

        # create topleft_coordinate using both pos & neg
        self.topleft_coordinate = []
        self.topleft_coordinate.extend (pos_topleft_coord)
        self.topleft_coordinate.extend (neg_topleft_coord)
        self.topleft_coordinate = np.array(self.topleft_coordinate)
        np.random.shuffle (self.topleft_coordinate)
        
        self.size = self.topleft_coordinate.shape[0]
        
        if normalization: 
            self.normalization = normalization
            assert normalization in set(['mean', 'Gaussian'])
            self._cal_norm_param()

    def _cal_norm_param(self):
        mu = 0
        img_size = self.img_size
        # Careful! norm params not yet calculated
        for patch in self.iterate_raw_image_patches(norm = False):
            mu += patch[0]
            assert (patch != -9999).all()
        mu = (mu / self.size).mean(axis=(1,2))
        self.mu = mu
        print("mu = ", mu)

        if self.normalization == 'Gaussian':
            std = 0
            mu_ext = np.repeat(mu, [np.prod(patch[0][0].shape)]*patch[0].shape[0]).reshape(patch[0].shape)
            
            for patch in self.iterate_raw_image_patches(norm = False):
                std += ((patch[0]-mu_ext)**2).mean(axis=(-1,-2))
            std = np.sqrt(std / self.size)
            self.std = std
            print('std = ', std)



    def _get_raw_patch(self, coord, norm):
        patch  = self.raw_image[:, coord[0]:coord[0]+self.img_size, coord[1]:coord[1]+self.img_size]
        if norm:
            if self.normalization == 'mean':
                for channel_num in range(self.band):
                    patch[channel_num] = patch[channel_num] - self.mu[channel_num]
            else:
                for channel_num in range(self.band):
                    patch[channel_num] = (patch[channel_num] - self.mu[channel_num]) / self.std[channel_num]
        return patch

    def _get_patch_label(self, coord):
        label = self.road_mask[int(coord[0]+self.img_size/2), int(coord[1]+self.img_size/2)]
        if self.encoding == 'one-hot':
            one_hot_label = np.zeros(2)
            one_hot_label[label] = 1
            return one_hot_label
        return label

    def iterate_raw_image_patches (self, norm):
        for coord in self.topleft_coordinate:
            yield np.array([self._get_raw_patch(coord, norm)])
                
    def iterate_raw_image_patches_with_coord (self, norm):
        for coord in self.topleft_coordinate:
            yield coord, np.array([self._get_raw_patch(coord, norm)])

    def iterate_data (self, norm=True):
        for coord in self.topleft_coordinate:
            x = np.array([self._get_raw_patch(coord, norm)])
            y = np.array([self._get_patch_label(coord)])
            yield x, y

    def iterate_data_with_coord (self, norm=True):
        for coord in self.topleft_coordinate:
            x = np.array([self._get_raw_patch(coord, norm)])
            y = np.array([self._get_patch_label(coord)])
            yield coord, x, y


    def _get_patches_from_topleft_coord (self, coord_arr, num_of_patches, start_index, norm, wrap_around):
        X = []
        Y = []
        
        start = start_index
        end = start_index + num_of_patches
        compensation = 0
            
        # check overflow
        if end > coord_arr.shape[0]:
            compensation = end - coord_arr.shape[0]
            end = coord_arr.shape[0]
        
        # appending
        for idx in range(start, end):
            coord = coord_arr [idx]

            X.append(self._get_raw_patch(coord, norm))
            Y.append(self._get_patch_label(coord))

        start_index = end

        # wrap around
        if compensation > 0:
            np.random.shuffle (coord_arr)

            if wrap_around:
                end = compensation
                for idx in range(0, end):
                    coord = coord_arr [idx]
                    
                    X.append(self._get_raw_patch(coord, norm))
                    Y.append(self._get_patch_label(coord))

                start_index = end

            else:
                start_index = 0            

        return X, Y, start_index
    
    # top-left coordinate should be of shape (n, 2)
    def get_patches(self, batch_size, positive_num = 0, norm = True, wrap_around=True):
        X = []
        Y = []
        
        if positive_num > 0 and positive_num <= batch_size:
            # pos patches
            X_pos, Y_pos, self.pos_index = self._get_patches_from_topleft_coord (self.pos_topleft_coord,
                                                                     num_of_patches = positive_num,
                                                                     start_index = self.pos_index,
                                                                     norm = norm, 
                                                                     wrap_around=wrap_around)
            
            negative_num = batch_size - positive_num
            X_neg, Y_neg, self.neg_index = self._get_patches_from_topleft_coord (self.neg_topleft_coord,
                                                                     num_of_patches = negative_num,
                                                                     start_index = self.neg_index,
                                                                     norm = norm, 
                                                                     wrap_around=wrap_around)
            
            X.extend (X_pos)
            Y.extend (Y_pos)
            
            X.extend (X_neg)
            Y.extend (Y_neg)
        
        else: # sample batches randomly
            X, Y, self.index = self._get_patches_from_topleft_coord (self.topleft_coordinate,
                                                                      num_of_patches = batch_size,
                                                                      start_index = self.index,
                                                                      norm = norm,
                                                                      wrap_around=wrap_around)

        X = np.array(X)
        Y = np.array(Y)
        return X, Y

In [12]:
CV_Data = Data_Extractor (CV_raw_image, CV_road_mask, 8,
                          pos_topleft_coord = CV_pos_topleft_coord,
                          neg_topleft_coord = CV_neg_topleft_coord, 
                          normalization='Gaussian')

mu =  [ 150.8227309   198.28994142  365.46753706  373.66751856 2370.63525823
 1356.64782764  731.00466191]
std =  [126.38246319 147.99541069 225.10661372 260.4291769  792.72602026
 819.12928051 548.31700647]


In [13]:
CV_Data.mu.shape

(7,)

In [30]:
class FCN_Data_Extractor (Data_Extractor):

    def __init__(self, raw_image, road_mask, img_size, pos_topleft_coord, neg_topleft_coord, normalization='mean', encoding='one-hot'):

        super(FCN_Data_Extractor, self).__init__(raw_image, road_mask, img_size, pos_topleft_coord, neg_topleft_coord, normalization, encoding)
        self.neg_weight = self.pos_size / self.size
        self.pos_weight = self.neg_size / self.size
        
    def _get_patch_label(self, coord):
        label = self.road_mask[coord[0]:coord[0]+self.img_size, coord[1]:coord[1]+self.img_size]
        if self.encoding == 'one-hot':
            one_hot_label = np.zeros((self.img_size, self.img_size, 2))
            one_hot_label[:, :, 0][np.where(label == 0)] = 1
            one_hot_label[:, :, 1][np.where(label == 1)] = 1
            return one_hot_label
        return label

    def get_patches(self, batch_size, positive_num = 0, norm = True, wrap_around=True, weighted=True):
        X = []
        Y = []

        if positive_num > 0 and positive_num <= batch_size:
            # pos patches
            X_pos, Y_pos, self.pos_index = self._get_patches_from_topleft_coord (self.pos_topleft_coord,
                                                                     num_of_patches = positive_num,
                                                                     start_index = self.pos_index,
                                                                     norm = norm, 
                                                                     wrap_around=wrap_around)
            
            negative_num = batch_size - positive_num
            X_neg, Y_neg, self.neg_index = self._get_patches_from_topleft_coord (self.neg_topleft_coord,
                                                                     num_of_patches = negative_num,
                                                                     start_index = self.neg_index,
                                                                     norm = norm, 
                                                                     wrap_around=wrap_around)
            
            X.extend (X_pos)
            Y.extend (Y_pos)
            
            X.extend (X_neg)
            Y.extend (Y_neg)
        
        else: # sample batches randomly
            X, Y, self.index = self._get_patches_from_topleft_coord (self.topleft_coordinate,
                                                                      num_of_patches = batch_size,
                                                                      start_index = self.index,
                                                                      norm = norm,
                                                                      wrap_around=wrap_around)

        X = np.array(X)
        Y = np.array(Y)
    
        if weighted:
            Weight = Y.copy()
            Weight[:,:,:,0] *= self.neg_weight
            Weight[:,:,:,1] *= self.pos_weight
            Weight = Weight.sum(axis=-1)

        return X, Y, Weight

In [2]:
import h5py
import gc

path_train_set = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/posneg_topleft_coord_split_8_train"
path_cv_set = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert_uncl_track/posneg_topleft_coord_split_8_cv"

# Load training set
train_set = h5py.File(path_train_set, 'r')
train_pos_topleft_coord = np.array(train_set['positive_example'])
train_neg_topleft_coord = np.array(train_set['negative_example'])
train_raw_image = np.array(train_set['raw_image'])
train_road_mask = np.array(train_set['road_mask'])
train_set.close()

# Load cross-validation set
CV_set = h5py.File(path_cv_set, 'r')
CV_pos_topleft_coord = np.array(CV_set['positive_example'])
CV_neg_topleft_coord = np.array(CV_set['negative_example'])
CV_raw_image = np.array(CV_set['raw_image'])
CV_road_mask = np.array(CV_set['road_mask'])
CV_set.close()

gc.collect()

/home/admin-u6142160/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


0

In [32]:
X, Y = CV_Data.get_patches(2)
print(X.shape, Y.shape)

(2, 7, 8, 8) (2,)


In [33]:
CV_FCN_Data = FCN_Data_Extractor (CV_raw_image, CV_road_mask, 8,
                                  pos_topleft_coord = CV_pos_topleft_coord,
                                  neg_topleft_coord = CV_neg_topleft_coord)

In [34]:
x, y, w = CV_FCN_Data.get_patches(2)
print(x.shape, y.shape, w.shape)

(2, 7, 8, 8) (2, 8, 8, 2) (2, 8, 8)


In [38]:

y_f = y.reshape(-1,2)
w_f = w.reshape(-1)
for i in range(y_f.shape[0]):
    print(y_f[i], w_f[i])
    assert sum(np.array([CV_FCN_Data.neg_weight, CV_FCN_Data.pos_weight]) * y_f[i]) == w_f[i] 